In [3]:
# tokenize the whole Compact Memory dataset, page-wise, using Spacy
# each page is dumped as a dictionary with fields 'lang', 'tokens'
import os, json, pickle, pprint
import spacy
from langdetect import detect, lang_detect_exception
root_data = "/data/cm/"
root = "/home/rovera/cm/"

In [4]:
nlp = spacy.load("de_core_news_lg")

In [5]:
def process_doc(text):
    tc, wc = 0, 0 # token, word count
    # output dict
    out = {'lang': None, 'tokens': [], 'wc': 0, 'tc': 0}
    # detect language
    out['lang'] = get_language(text)
    
    # process the doc, this requires to have nlp = spacy.load('model_name') somewhere in the main flow ;)
    doc = nlp(text)
    
    for token in doc:
        tc += 1 
        word = token.text
        out['tokens'].append(word)
        
        if not token.is_punct and not token.is_stop:
            wc += 1
    
    out['wc'] = wc 
    out['tc'] = tc 
    
    
    return out
    

def get_language(text):
    lang = None
    try:
        lang = detect(text)
    except lang_detect_exception.LangDetectException:
        pass
    
    return lang

In [5]:
# NOT TO REUSE, IT OVERWRITES!!!
# create a list of journals in order to keep track of which journals have been processed
#proc_journals = {}
#for j in os.listdir(root_data+"output/text/"):
#    proc_journals[j] = False
#with open(root_data+"output/nlp_pipeline/tokenized_CM_indexes/journals_status.json", "w", encoding="utf-8") as outfile:
#    json.dump(proc_journals, outfile, indent=2)

In [6]:
# NOT TO REUSE, IT OVERWRITES!!!
# first iterate over the journals and collect the pages to be processed
# create
# a) an output folder named according to the journal id that will contain a file with entities for each page
# b) an index for each journal that describes the processing status of each page
"""
for journal in os.listdir(root_data+"output/text/"):
        
    print("Processing {}...".format(journal), end="")
        
    # create an output folder for the journal
    if not os.path.exists(root_data+"output/nlp_pipeline/tokenized_CM/"+journal):
        os.makedirs(root_data+"output/nlp_pipeline/tokenized_CM/"+journal)
        
    # create and index file for that journal
    journal_index = {}
        
    for (rs, ds, fs) in os.walk(root_data+"output/text/"+journal+"/"):
            
        if fs != []:
            pathname = rs.split('text/')[1].replace('/', '_')

            for file in fs:
                curr_filename = file.split('.')[0]
                output_filename = pathname + "__" + curr_filename
                    
                journal_index[output_filename] = False
                    
    with open(root_data+"output/nlp_pipeline/tokenized_CM_indexes/journals/"+journal+".json", "w", encoding="utf-8") as outfile:
        json.dump(journal_index, outfile, ensure_ascii=False, indent=2)
        
    print("...Done.")
"""

Processing 2710055......Done.
Processing 9616701......Done.
Processing 2580773......Done.
Processing 2895450......Done.
Processing 2727810......Done.
Processing 3129675......Done.
Processing 6492429......Done.
Processing 2438141......Done.
Processing 2827798......Done.
Processing 5438908......Done.
Processing 4782723......Done.
Processing 7506414......Done.
Processing 9054153......Done.
Processing 9366335......Done.
Processing 4889892......Done.
Processing 2425321......Done.
Processing 6492329......Done.
Processing 700673......Done.
Processing 4895413......Done.
Processing 2542917......Done.
Processing 9583346......Done.
Processing 2549643......Done.
Processing 10638461......Done.
Processing 3139318......Done.
Processing 9620169......Done.
Processing 9051331......Done.
Processing 4781937......Done.
Processing 4911569......Done.
Processing 2625941......Done.
Processing 2446951......Done.
Processing 9643693......Done.
Processing 2912821......Done.
Processing 6465881......Done.
Processing

Processing 1304829......Done.
Processing 9607246......Done.
Processing 6492399......Done.
Processing 3111067......Done.
Processing 2420797......Done.
Processing 4804947......Done.
Processing 2609281......Done.
Processing 3122061......Done.
Processing 2424658......Done.
Processing 4926451......Done.
Processing 2571117......Done.
Processing 2708691......Done.
Processing 8812431......Done.
Processing 9215017......Done.
Processing 2420973......Done.
Processing 2550047......Done.
Processing 4926100......Done.
Processing 3138070......Done.
Processing 4862473......Done.
Processing 8003959......Done.
Processing 2821022......Done.
Processing 2416673......Done.


In [ ]:
journals_status = json.load(open(root_data+"output/nlp_pipeline/tokenized_CM_indexes/journals_status.json", 'r', encoding="utf-8"))

for journal, status in journals_status.items():
        
    if status is False:
        
        print("Processing journal {}...".format(journal))
        
        c = 0
        
        # open corresponding journal index
        journal_index = json.load(open(root_data+"output/nlp_pipeline/tokenized_CM_indexes/journals/"+journal+".json", "r"))
        
        tot = len(journal_index)
        to_process = len([k for k,v in journal_index.items() if v is False])
        
        print("{} total pages in this journal.".format(str(tot)))
        print("{} pages still to process.".format(str(to_process)))
        
        for page, page_status in journal_index.items():
            
            if page_status is False:
                
                # open corresponding file for processing
                pagepath, pagename = page.split("__")
                pagepath = pagepath.replace('_', '/') + '/'
                
                pagename = pagename.lstrip("_")
                
                text = open(root_data+"output/text/"+pagepath+pagename+".txt", "r", encoding="utf-8").read()
                res = process_doc(text)
                
                # save the result
                json.dump(res, open(root_data+"output/nlp_pipeline/tokenized_CM/"+journal+"/"+page+".json", "w", encoding="utf-8"), ensure_ascii=False, indent=2)
                # update the index with status for the current page
                journal_index[page] = True
                # save back updated journal index
                with open(root_data+"output/nlp_pipeline/tokenized_CM_indexes/journals/"+journal+".json", "w") as outfile:
                    json.dump(journal_index, outfile, indent=2)
            c += 1
            if c % 100 == 0:
                print("|", end="")
        # update journal status to True
        journals_status[journal] = True
        with open(root_data+"output/nlp_pipeline/tokenized_CM_indexes/journals_status.json", "w") as o_file:
            json.dump(journals_status, o_file, indent=2)
        print(" Done.")
        print()
    else:
        print("Journal {} has already been processed.".format(journal))

Processing journal 2710055...
130 total pages in this journal.
130 pages still to process.
| Done.

Processing journal 9616701...
2391 total pages in this journal.
2391 pages still to process.
||||||||||||||||||||||| Done.

Processing journal 2580773...
2686 total pages in this journal.
2686 pages still to process.
|||||||||||||||||||||||||| Done.

Processing journal 2895450...
8317 total pages in this journal.
8317 pages still to process.
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| Done.

Processing journal 2727810...
4823 total pages in this journal.
4823 pages still to process.
|||||||||||||||||||||||||||||||||||||||||||||||| Done.

Processing journal 3129675...
3208 total pages in this journal.
3208 pages still to process.
|||||||||||||||||||||||||||||||| Done.

Processing journal 6492429...
973 total pages in this journal.
973 pages still to process.
||||||||| Done.

Processing journal 2438141...
6156 total pages in this journal.
6156 pages 